<a href="https://colab.research.google.com/github/16Komang/Big-Data-Tugas-7/blob/main/slide48.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install PySpark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=d4b2f7ef23aca103739145f601f29baf1daaa26f141dff1e42e6e29849abecdb
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
from tabulate import tabulate
from pyspark.mllib.recommendation import ALS, Rating
from pyspark.sql import SparkSession

In [ ]:
# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
# Read the ratings data from a file
raw = spark.sparkContext.textFile("/content/drive/MyDrive/Big data/ml-1m/ratings.dat")
mydata = [(2, 0.01), (3, 0.02), (4, 0.03)]
mydatardd = spark.sparkContext.parallelize(mydata).map(lambda x: Rating(0, x[0], x[1]))

In [ ]:
# Define a function to parse the rating data
def parseRating(str):
    fields = str.split("::")
    assert len(fields) == 4
    return Rating(int(fields[0]), int(fields[1]), float(fields[2]))

In [ ]:
# Parse the rating data
ratings = raw.map(parseRating)
totalRatings = ratings.union(mydatardd)

In [ ]:
# Train the ALS model
rank = 8
numIterations = 5
lambda_ = 1.0
model = ALS.train(totalRatings, rank, numIterations, lambda_)

In [ ]:
# Get the top 10 recommended products for user 1
userId = 1
numRecommendations = 10
products = model.recommendProducts(userId, numRecommendations)

In [ ]:
# Convert the products list to an RDD
productsRDD = spark.sparkContext.parallelize(products)

In [ ]:
# Load movie data, join with the recommendations, and display the names ordered by ratings
movies = spark.read.text("/content/drive/MyDrive/Big data/ml-1m/movies.dat")
movieNames = movies.rdd.map(lambda line: line.value.split("::")).map(lambda fields: (int(fields[0]), fields[1]))
recommendations = productsRDD.map(lambda rating: (rating.product, rating.rating))
movieRecommendations = movieNames.join(recommendations).sortBy(lambda x: -x[1][1])

# Prepare the data for tabular display
table_data = [(movie_id, movie_title, rating) for movie_id, (movie_title, rating) in movieRecommendations.take(10)]

# Display the recommendations in a table
headers = ["Movie ID", "Title", "Rating"]
table = tabulate(table_data, headers, tablefmt="grid")
print(table)

+------------+-------------------------------------------+----------+
|   Movie ID | Title                                     |   Rating |
+============+===========================================+==========+
|       3382 | Song of Freedom (1936)                    |  4.4463  |
+------------+-------------------------------------------+----------+
|        989 | Schlafes Bruder (Brother of Sleep) (1995) |  4.02357 |
+------------+-------------------------------------------+----------+
|       3233 | Smashing Time (1967)                      |  3.90453 |
+------------+-------------------------------------------+----------+
|        787 | Gate of Heavenly Peace, The (1995)        |  3.9044  |
+------------+-------------------------------------------+----------+
|        557 | Mamma Roma (1962)                         |  3.90174 |
+------------+-------------------------------------------+----------+
|       1830 | Follow the Bitch (1998)                   |  3.90091 |
+------------+------